# OOP: class variables (klasės kintamieji)

## The Concepts of Objectively Oriented Programming

* **Klasė** (class) − vartotojo apibrėžtas objekto prototipas, kuris apibrėžia atributų, charakterizuojančių bet kurį tos klasės objektą, rinkinį.  
* **Atributai** (attributes) - tai duomenų atstovai (klasės kintamieji arba egzempliorių kintamieji) ir metodai, prieinami naudojant taško žymėjimą.  

* **Klasės kintamasis** (class variable) − kintamasis, priklausantis visiems tos klasės egzemplioriams. Klasės kintamieji yra apibrėžiami klasės viduje, bet už metodų ribų.  

* **Egzemplioriaus kintamasis** (instance variable) − kintamasis, apibrėžiamas tam tikrame klasės metode ir priklausantis tik tam tikram klasės instance.  

* **Metodas** (method) - tai tam tikra funkcija, apibrėžiama klasės apraše.  

* **Duomenų atstovas** (data member) − tai klasės kintamasis arba egzemplioriaus kintamasis, kuriame laikomi duomenys, susieti su klase ir jos objektais.  

* **Egzempliorius** (instance) − individualus tam tikros klasės objektas.    

* **Instantiation / initialization** − klasės egzemplioriaus sukūrimas.  

* **Objektas** (object) − unikalus duomenų struktūros egzempliorius, apibrėžtas klasės, kuriam jis priklauso. Objektus sudaro duomenų atstovai (klasės kintamieji arba instance kintamieji) ir metodai.  

[Versta iš [TutorialsPoint](https://www.tutorialspoint.com/python/python_classes_objects.htm)]

## Atributų rūšys
Kiekvienas objektas turi tik dviejų tipų atributus: duomenų atstovus ir metodus. Duomenų atstovų negalima iškviesti (jei jų reikšmės nėra taip pat ir metodai), o metodus galima:

In [72]:
import numpy as np
arr = np.arange(10)
arr

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Duomenų atstovo pavyzdys:

In [73]:
arr.itemsize

8

Metodo pavyzdys:

In [80]:
arr.max()

9

Kas būna, jei bandome iškviesti duomenų atstovą:

In [81]:
arr.itemsize()

TypeError: 'int' object is not callable

Duomenų atstovo (data member) terminas programavime yra vartojamas rečiau. Todėl šioje temoje jį vadinsime tiesiog atributu, turėdami mintyje, kad čia jų sintaksė bus `obj.attr`, bet ne `obj.attr()`

## Nesaugus attributų keitimas

### Nepageidautinos objekto elgsenos pavyzdys

Imkime tam tikrą Python objektą ir pasvarstykime, ar jam galėtume priskirti atributus:

In [82]:
arr = np.array([8, 3, 4], dtype=np.uint16)
arr.strides

(2,)

In [83]:
arr.strides = 1
arr

array([  8, 768,   3], dtype=uint16)

In [65]:
np.lib.stride_tricks.as_strided(arr, shape=(3,), strides=(1,))

array([  8, 768,   3], dtype=uint16)

Tai buvo nuotaikingas pavyzdys, kaip buvo galima nepageidaujamai pakeisti objekto elgseną. Iš tiesų, kiekvienas masyvo narys buvo saugomas 16 bitų ilgio atmintyje: 

In [57]:
arr_bits = np.unpackbits(np.array([8, 3, 4], dtype='>i2').view(np.uint8)).reshape(-1, 16)
arr_bits

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]], dtype=uint8)

Neatsakingai naudodami atributą `.strides`, mes pakeitėme jo reikšmę, kuri nurodo, kiek baitų (po 8 bitus) reikia praleisti, norint nuskaityti masyvo duomenis. Gavome, kad dabar masyvo nariai buvo skaitomi iš tokių bitų eilučių:

In [67]:
np.lib.stride_tricks.as_strided(arr_bits.ravel(), shape=(3, 16), strides=(8, 1))

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]], dtype=uint8)

Todėl rezultatas buvo trys skaičiai: 8, 768 ir 3

### Neleistino atributo keitimo pavyzdys

Žinome, kad masyvai turi atributą `.nbytes`, parodantį, kiek baitų vietos jis užima

In [70]:
arr = np.array([8, 3, 4], dtype=np.uint16)
arr.nbytes

6

Štai, kas bus, jei bandysime jį pakeisti

In [71]:
arr.nbytes = 3

AttributeError: attribute 'nbytes' of 'numpy.ndarray' objects is not writable

### Neegzistuojančio atributo pavyzdys

In [84]:
arr = np.array([8, 3, 4], dtype=np.uint16)
arr.party = 5

AttributeError: 'numpy.ndarray' object has no attribute 'party'

Visais šiais atvejais (nepageidautinos objekto elgsenos, neleistino atributo keitimo ir neegzistuojančio atributo keitimo) atributo keitimas nebuvo sklandus. Todėl reikia ieškoti kitokių sprendimų.

## Saugus atributų keitimas

Galima įsitikinti, kad masyvai taip pat turi atributą `flags`:

In [3]:
arr = np.array([8, 3, 4], dtype=np.uint16)
arr.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

Šis atributas yra Python žodynas. Galime pakeisti vieną iš to žodyno elementų:

In [16]:
arr.flags['WRITEABLE'] = False

Tokiu būdu pakeisime šio objekto nustatymus taip, kad į jį nebūtų galima įrašyti duomenų:

In [17]:
arr[1] = 99

ValueError: assignment destination is read-only

Tai vis dar nėra saugus atributo keitimo būdas. Įprastai tokiais atvejais objektai turėtų būti apibrėžiami taip, kad turėtų metodus, skirtus keisti atributams

In [18]:
arr.setflags(write=True)
arr[1] = 99
arr

array([ 8, 99,  4], dtype=uint16)

[Ar tokį metodą turi `arr.strides` atributas](https://stackoverflow.com/questions/71166797)?

## Kaip sukurti objektą su norimais atributais? 

Prieš tai ypatingą dėmesį skyrėme masyvams, kurie yra klasės `numpy.ndarray` objektai:

In [26]:
print(type(arr))

<class 'numpy.ndarray'>


Jų elgsena buvo aprašyta `numpy` bibliotekos kūrėjų. 

### Kuriame paprasčiausią objektą, turintį atributus
Tarkime dabar norime sukurti objektą, kurio pageidaujamos savybės yra tokios:

* objektas turi duomenų atstovą `age`
* objektas turi duomenų atstovą `weight`
* objektas turi duomenų atstovą `height`
* objektas apibūdina moksleivį

Tokiu atveju klasę turime aprašyti patys:

In [70]:
class Student:
    age = 14
    weight = 68
    height = 172

Marius = Student()

Čia buvo sukurtas `Student` klasės objektas `Marius`. Pažiūrėję šio objekto tipą pamatysime, kad jis susideda iš dviejų dalių: aplinkos, kurioje buvo kuriama klasė, pavadinimo, ir klasės pavadinimo:

In [30]:
print(type(Marius))

<class '__main__.Student'>


Mūsų sukurtas objektas jau turės atributus, kurių pageidavome:

In [32]:
Marius.age, Marius.weight, Marius.height

(12, 68, 172)

Juos taip pat galima keisti:

In [34]:
Marius.age = 15
Marius.weight = 70
Marius.height = 175

In [35]:
Marius.age, Marius.weight, Marius.height

(15, 70, 175)

Dabar sukurkime kitą objektą vardu Aldona, kuri yra jaunesnė ir mažesnio sudėjimo:

In [67]:
Aldona = Student()
Aldona.age, Aldona.weight, Aldona.height

(14, 68, 172)

In [69]:
Aldona.age, Aldona.weight, Aldona.height = 12, 49, 145
Aldona.age, Aldona.weight, Aldona.height

(12, 49, 145)

Matome, kad sukūrus kiekvieną `Student` klasės objektą, jo atributai yra nustatomi automatiškai. Tai dažniausiai nėra pageidautina, nes automatiškai sukurti neteisingas atributų reikšmes ir paskui jas perrašyti nei efektyvu, nei patrauklu vartotojui. Be to, būdas kuriuo priskyrėme atributų reikšmes, bendru atveju nėra saugus, ne objekto elgsena gali pasikeisti netikėta linkme.

Kitoje temoje apžvelgsime būdus, kaip galima pakeisti objekto `Student` aprašą į tinkamesnį. O kol kas atkreipkime dėmesį, kad dabartiniame klasės apraše nebuvo nei vieno metodo. Vadinasi, visi kintamieji, kuriuos apibrėžėme, buvo klasės, bet ne egzemplioriaus kintamieji. Gerokai dažniau pasitaiko klasės, kuriose kuriami egzemplioriaus kintamieji. Apie tai - kitoje temoje.